# Problem 1 (10 pts):

From Maximum Likelihood to Cross-Entropy Loss
Learning Objectives: Connect probability theory to loss functions, understand why cross-entropy emerges naturally.

Part A: Binary Classification Loss Derivation

Setup: We have $n$ data points $\{ (x_i, y_i) \}_{i=1}^n$ where $x_i \in \mathbb{R}^d$ and $y_i \in \{ 0, 1 \}$. Assume your model outputs the probability of class 1 as $p_i = p(y_i = 1 \mid x_i) = \sigma(w^T x_i + b)$ where $w \in \mathbb{R}^d$, $b \in \mathbb{R}$, and $\sigma(z)$ is the sigmoid function $\sigma(z) = 1 / (1 + e^{-z})$.

1. Derive from MLE:
    * Write the likelihood function for the dataset
    * Take the log-likelihood
    * Show that maximizing log-likelihood = minimizing binary cross-entropy
    * Bonus (5 pts): Derive the gradient and show it has the nice form: $\nabla_w BCE = X^T(p - y)$

Part B: Extension to Multi-class

1. Softmax derivation: Extend to $K$ classes using softmax function
    * Likelihood
    * Log-likelihood
    * Maximizing log-likelihood = minimizing categorical cross-entropy

Part C: Implement dependencies in `hw1_impl.py` for function `problem_1_part_c` in `hw1_script.py`. You will implement both binary and multi-class cross-entropy from scratch. You will compare your implementation with `sklearn.metrics.log_loss`.

## Answer: Problem 1, Part A
## 1. Likelihood Function

Each $y_i$ follows a Bernoulli distribution with parameter $p_i$:

$$
p(y_i \mid x_i) = p_i^{y_i}(1 - p_i)^{1 - y_i}.
$$

Assuming i.i.d. data, the likelihood over the dataset is:

$$
\mathcal{L}(w,b)
=
\prod_{i=1}^n
p_i^{y_i}(1 - p_i)^{1 - y_i}.
$$

---

## 2. Log-Likelihood

Taking logs:

$$
\log \mathcal{L}(w,b)
=
\sum_{i=1}^n
\left[
y_i \log p_i
+
(1 - y_i)\log(1 - p_i)
\right].
$$

---

## 3. Maximizing Log-Likelihood = Minimizing Binary Cross-Entropy

Maximizing $\log \mathcal{L}$ is equivalent to minimizing the negative log-likelihood:

$$
-\log \mathcal{L}(w,b)
=
\sum_{i=1}^n
\left[
- y_i \log p_i
-
(1 - y_i)\log(1 - p_i)
\right].
$$

This is the same as **binary cross-entropy (BCE)** objective.

$$
\mathrm{BCE}(w,b)
=
\frac{1}{n}
\sum_{i=1}^n
\left[
- y_i \log p_i
-
(1 - y_i)\log(1 - p_i)
\right].
$$

Therefore,

$$
\arg\max_{w,b} \log \mathcal{L}(w,b)
=
\arg\min_{w,b} \mathrm{BCE}(w,b).
$$

## Bonus — Gradient Derivation

Define:

- $z_i = w^\top x_i + b$
- $p_i = \sigma(z_i)$

Let:

- $X \in \mathbb{R}^{n \times d}$ contain rows $x_i^\top$
- $y \in \mathbb{R}^n$ contain entries $y_i$
- $p \in \mathbb{R}^n$ contain entries $p_i$

Using the sum version of BCE:

$$
\mathrm{BCE}_{\text{sum}}
=
\sum_{i=1}^n
\left[
- y_i \log p_i
-
(1 - y_i)\log(1 - p_i)
\right].
$$

A key simplification occurs:

$$
\frac{\partial}{\partial z_i}
=
p_i - y_i.
$$

Using the chain rule $\frac{\partial z_i}{\partial w} = x_i$, we obtain:

$$
\nabla_w \mathrm{BCE}_{\text{sum}}
=
\sum_{i=1}^n (p_i - y_i) x_i
=
X^\top (p - y).
$$

If using the mean BCE:

$$
\nabla_w \mathrm{BCE}
=
\frac{1}{n} X^\top (p - y).
$$

Similarly,

$$
\frac{\partial \mathrm{BCE}_{\text{sum}}}{\partial b}
=
\sum_{i=1}^n (p_i - y_i).
$$


## Answer:  Problem 1, Part B

Now extend to $K$ classes.

Each label is one-hot encoded:

$$
y_i \in \{0,1\}^K,
\quad
\sum_{k=1}^K y_{ik} = 1.
$$

Model parameters:

- $W \in \mathbb{R}^{d \times K}$
- $b \in \mathbb{R}^K$

Logits:

$$
o_i = W^\top x_i + b,
\quad
o_{ik} = (W^\top x_i + b)_k.
$$

---

## Softmax Probabilities

$$
p_{ik}
=
\frac{\exp(o_{ik})}
{\sum_{j=1}^K \exp(o_{ij})}.
$$

---

## 1. Likelihood

For one example:

$$
p(y_i \mid x_i)
=
\prod_{k=1}^K p_{ik}^{y_{ik}}.
$$

Dataset likelihood:

$$
\mathcal{L}(W,b)
=
\prod_{i=1}^n
\prod_{k=1}^K
p_{ik}^{y_{ik}}.
$$

---

## 2. Log-Likelihood

$$
\log \mathcal{L}(W,b)
=
\sum_{i=1}^n
\sum_{k=1}^K
y_{ik} \log p_{ik}.
$$

---

## 3. Maximizing Log-Likelihood = Minimizing Categorical Cross-Entropy

Negative log-likelihood:

$$
-\log \mathcal{L}(W,b)
=
\sum_{i=1}^n
\sum_{k=1}^K
- y_{ik} \log p_{ik}.
$$

This is the **categorical cross-entropy** loss:

$$
\mathrm{CE}(W,b)
=
\frac{1}{n}
\sum_{i=1}^n
\sum_{k=1}^K
- y_{ik} \log p_{ik}.
$$

Therefore,

$$
\arg\max_{W,b} \log \mathcal{L}(W,b)
=
\arg\min_{W,b} \mathrm{CE}(W,b).
$$


# Problem 2 (10 pts): Normal Equations vs. Gradient Descent - A Computational Study
Learning Objectives: Understand trade-offs between analytical and iterative solutions.

Implement dependencies in `hw1_impl.py` for function `problem_2` in `hw1_script.py`.

Analysis Tasks:

1. Answer this question. Memory Usage: When does the normal equation become impractical?
2. Conditioning: What happens when $X^TX$ is nearly singular? Add ridge regularization.
3. Report: When would you choose each method in practice?

## Answer: Problem 2

## 1) Memory Usage — When does the normal equation become impractical?

The NE method forms and solves the system
$$
(X_{\text{aug}}^\top X_{\text{aug}}) W = X_{\text{aug}}^\top Y,
$$
where
$$
X_{\text{aug}}^\top X_{\text{aug}} \in \mathbb{R}^{(d+1)\times(d+1)}.
$$

Key scaling facts (dense case):

- **Memory** to store $X_{\text{aug}}^\top X_{\text{aug}}$ is $O(d^2)$.
- **Time** to form $X_{\text{aug}}^\top X_{\text{aug}}$ is $O(nd^2)$.
- **Time** to solve the linear system is $O(d^3)$ (typical direct methods).

Therefore, the normal equation becomes impractical primarily when the **feature dimension $d$ is large**, because:
- storing a dense $(d+1)\times(d+1)$ matrix becomes memory-heavy,
- the $O(d^3)$ solve becomes prohibitively slow.

By contrast, GD avoids $d^2$ storage and instead uses repeated matrix–vector/matrix–matrix products:
- each iteration costs roughly $O(ndm)$,
- memory is dominated by storing $X_{\text{aug}}$ and $W$ (and possibly history).

---

## 2) Conditioning — What happens when $X^\top X$ is nearly singular? Add ridge regularization.

When $X_{\text{aug}}^\top X_{\text{aug}}$ is **singular** or **ill-conditioned** (nearly singular), this typically occurs due to:
- multicollinearity (highly correlated features),
- redundant features (linear dependence),
- $d \ge n$ (rank deficiency),
- low effective rank.

Consequences for NE:
- solving becomes numerically unstable,
- small perturbations in data can produce large changes in $W$,
- coefficients can blow up, harming generalization.

### Ridge regularization (as implemented)

Ridge modifies the system to:
$$
\left(X_{\text{aug}}^\top X_{\text{aug}} + \lambda R\right) W = X_{\text{aug}}^\top Y,
$$
where $R$ is identity except bias is not regularized:
$$
R = I_{d+1}, \quad R_{00}=0.
$$

Effects:
- adds $\lambda$ to the eigenvalues associated with weight directions (not the bias),
- makes the matrix better conditioned (and typically invertible),
- shrinks weights, reducing variance and improving stability.

---

## 3) When would you choose each method in practice?

### Choose Normal Equations (NE) when:
- $d$ is small to moderate (so $d^2$ memory and $d^3$ time are manageable),
- you want a one-shot, deterministic solve,
- conditioning is acceptable, or ridge is applied,
- you can afford forming $X_{\text{aug}}^\top X_{\text{aug}}$.

### Choose Gradient Descent (GD) when:
- $d$ is large (NE becomes memory/time prohibitive),
- $n$ is very large and you want iterative/streaming optimization,
- the data is sparse (GD can exploit sparsity better than dense $X^\top X$),
- you anticipate extending beyond linear models (same optimization machinery applies),
- you can tolerate approximate solutions and control trade-offs via iterations and learning rate.

---

## Core Trade-off Summary

- NE is an **analytic/direct** approach: fast and exact for small $d$, but scales poorly with $d$ due to $O(d^2)$ memory and $O(d^3)$ solve time.
- GD is an **iterative/computational** approach: scales to large problems via repeated $O(nd)$-type operations, but depends on convergence behavior (learning rate, iterations, conditioning).
- Ill-conditioning harms both methods, but ridge regularization directly stabilizes NE (and typically improves optimization geometry for GD as well).

# Problem 3 (10 pts): SGD Exploration - Escaping Local Minima (Extended)
Learning Objectives: Understand SGD's stochastic nature and hyperparameter effects.

Due to history, there is no Part A.

Part B: Implement dependencies in `hw1_impl.py` for functions `problem_3_part_b` and `problem_3_part_c` in `hw1_script.py`.

Part D: Analysis Questions

1. What batch size gives the best exploration vs. exploitation trade-off?
2. How does the "escape probability" change with learning rate?

## Problem 3, Part D

We ignore Parts A–C per the prompt and focus only on the two analysis questions. The context is the “two-hole” (and optionally multi-modal) **loss landscape in parameter space** with SGD-like updates, where stochasticity is simulated via additive noise and an “escape” mechanism.

In the provided implementation, the SGD step is:
$$
w \leftarrow w - \text{lr}\,\tilde{g}(w),
$$
where the stochastic gradient is modeled as
$$
\tilde{g}(w) =
\begin{cases}
g_{\text{global}}(w) + \epsilon & \text{with probability } \text{escape\_chance},\\[4pt]
g_{\text{true}}(w) + \epsilon & \text{otherwise},
\end{cases}
$$
and
$$
g_{\text{true}}(w) = g_{\text{local}}(w) + g_{\text{global}}(w), 
\qquad
\epsilon \sim \mathcal{N}(0, \sigma^2 I).
$$

Critically, batch size affects the initial noise scale as implemented:
$$
\sigma \equiv \text{noise\_scale} = \frac{\text{initial\_noise}}{\sqrt{\text{batch\_size}}},
$$
and the noise decays over time:
$$
\sigma \leftarrow \sigma \cdot \text{noise\_decay} \quad \text{each iteration.}
$$

---

## D1) What batch size gives the best exploration vs. exploitation trade-off?

**Core relationship:** in this simulation, larger batch size reduces stochasticity:
$$
\text{batch\_size} \uparrow \;\Rightarrow\; \sigma \downarrow.
$$

### Small batch sizes (high noise)
- **Pros (exploration):** Higher $\sigma$ increases the chance of *escaping* shallow basins / local minima because random perturbations can push $w$ over “barriers” between attraction basins.
- **Cons (exploitation):** High noise prevents fine convergence; even near a minimum, updates jitter and can bounce out or hover without settling, especially if noise remains nontrivial.

### Large batch sizes (low noise)
- **Pros (exploitation):** Lower $\sigma$ yields stable, smooth descent; convergence is cleaner and more consistent once you’re in a good basin.
- **Cons (exploration):** Low noise makes escapes rare; trajectories become “deterministic” and can get trapped in the nearest local minimum determined by the start point and geometry.

### Best trade-off (answer)
The best exploration/exploitation balance is typically achieved by a **moderate batch size**:
- not so small that noise dominates and prevents convergence,
- not so large that the process becomes nearly deterministic and gets stuck.

In terms of the implemented scaling $\sigma \propto 1/\sqrt{\text{batch\_size}}$, “moderate” means a batch size that makes $\sigma$ large enough early on to cross basins, but small enough after decay to permit convergence. Practically, you’d see this in the heatmaps as:
- relatively **high escape probability**,
- **low best/mean loss** (finding the deeper minimum more often),
- **reasonable convergence iterations** (not exploding due to jitter),
- and runtime that’s not excessive.

So, **choose mid-range batch sizes** (neither extreme) as the best trade-off in this experiment.

---

## D2) How does the “escape probability” change with learning rate?

Here “escape probability” is measured empirically (e.g., `escaped = losses < thresh`) across trials. Learning rate affects escape in two opposing ways:

### Regime 1: Learning rate too small
- Steps are tiny:
  $$
  \Delta w \approx \text{lr}\,\tilde{g}(w)
  $$
- Even with noise, movement per iteration is limited.
- Result: it may take many steps to leave a basin; within a fixed iteration budget, **escape probability tends to be low**.

### Regime 2: Learning rate moderate
- Steps are large enough to traverse the landscape meaningfully.
- Noise + gradient can push parameters across basin boundaries.
- Result: **escape probability increases**, often substantially.

### Regime 3: Learning rate too large
- Updates can overshoot:
  - bounce across regions without settling,
  - become unstable and fail to converge,
  - or ricochet between areas, sometimes missing the global basin even if “escaping” the local one.
- Depending on the threshold definition, two outcomes occur:
  - **escape probability may plateau or drop** (if instability prevents reaching sufficiently low loss),
  - even if “movement” is large, **successful escapes to the deep minimum become less frequent**.

### Net effect (answer)
Escape probability typically shows a **non-monotonic** pattern with learning rate:
- **low** at very small lr,
- **higher** at moderate lr,
- **worse or unstable** at overly large lr (sometimes decreasing in “successful escape” rate).

In the heatmaps, this often appears as:
- moderate learning rates yielding the best combination of high escape probability and low best/mean loss,
- extreme high learning rates showing poor losses and inconsistent convergence iterations, even if trajectories move a lot.

---

## Summary

- **Best exploration vs. exploitation** in this setup comes from **moderate batch sizes**, because $\sigma \propto 1/\sqrt{\text{batch\_size}}$ makes small batches too noisy and large batches too deterministic.
- **Escape probability vs. learning rate** is typically **non-monotonic**: it rises from small lr to moderate lr, then worsens at overly large lr due to overshoot/instability and reduced “successful” convergence into the deep basin.

# Problem 4 (10 pts): The Perceptron Problem - Understanding Linear Separability Limitations

## What is a Perceptron?

Based on our lecture, a **perceptron** is a binary classifier that makes predictions using a linear decision boundary. It consists of:

- **Inputs**: A feature vector $x \in \mathbb{R}^d$
- **Weights**: A weight vector $w \in \mathbb{R}^d$
- **Bias**: A scalar bias term $b \in \mathbb{R}$
- **Activation**: A step function (threshold function)

The perceptron computes:
$$ f(x) = \text{step} (w^T x + b) $$

Where the step function outputs:
$$ \text{step}(w^T x + b) = \begin{cases}
1 & \text{if} \quad w^T x + b \geq 0 \\
0 & \text{if} \quad w^T x + b < 0
\end{cases} $$

The decision boundary is the hyperplane defined by $w^T x + b = 0$, which divides the input space into two regions.

## The Fundamental Problem

The perceptron suffers from a **critical limitation**: it can only solve **linearly separable** problems. This means it can only correctly classify data where the two classes can be perfectly separated by a single straight line (in 2D) or hyperplane (in higher dimensions).

### The XOR Problem: A Classic Example

The most famous demonstration of this limitation is the **XOR (Exclusive OR) problem**:

| x₁ | x₂ | XOR Output |
|----|----|------------|
| 0  | 0  | 0          |
| 0  | 1  | 1          |
| 1  | 0  | 1          |
| 1  | 1  | 0          |

If you plot these four points:
- Points (0,1) and (1,0) should be classified as class 1 (XOR = 1)
- Points (0,0) and (1,1) should be classified as class 0 (XOR = 0)

**No single straight line can separate these classes!** The pattern requires a non-linear decision boundary.

## Why This Matters

This limitation reveals why:

1. **Single perceptrons are insufficient** for many real-world problems
2. **We need non-linearity** in our models (like ReLU activation functions)
3. **Multiple layers are essential** to create complex, non-linear decision boundaries
4. **The XOR problem motivated** the development of multi-layer neural networks

As we learned in our previous lecture, when we combine multiple ReLU neurons and stack them in layers, we can create complex, bent decision boundaries that can solve non-linearly separable problems like XOR.

This historical limitation of the perceptron was so significant that it contributed to the "AI winter" of the 1970s, until researchers developed multi-layer networks with backpropagation in the 1980s.

Learning Objectives

1. Implement a perceptron from scratch to understand its mechanics
2. Demonstrate why linear models fail on non-linearly separable data
3. Visualize decision boundaries and their limitations
4. Show how adding non-linear features can solve the problem

Implement dependencies in `hw1_impl.py` for functions `problem_4` in `hw1_script.py`.